In [1]:
from dotenv import load_dotenv
import os

load_dotenv('../.env')


from typing import Annotated, TypedDict 
from langgraph.graph import  StateGraph, START, END

from langgraph.graph.message import add_messages 

from langchain_ollama import ChatOllama
from IPython.display import display, Image

llm = ChatOllama(model="llama3.2:3b", base_url = "http://localhost:11434")

llm.invoke("Hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-11-27T13:58:11.3518391Z', 'done': True, 'done_reason': 'stop', 'total_duration': 176807900, 'load_duration': 21172000, 'prompt_eval_count': 26, 'prompt_eval_duration': 28114000, 'eval_count': 8, 'eval_duration': 125811000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-fbf2e9ed-4590-4aa8-a531-3ab9f4ecab04-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [2]:
from datetime import datetime
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults



@tool
def internet_search(query: str):
    """
    Search the web for realtime and latest information.
    for examples, news, stock market, weather updates etc.
    
    Args:
    query: The search query
    """
    search = TavilySearchResults(
        max_results=3,
        search_depth='advanced',
        include_answer=True,
        include_raw_content=True,
    )

    response = search.invoke(query)

    return response

@tool
def llm_search(query: str):
    """
    Use the LLM model for general and basic information.
    """
    response = llm.invoke(query)
    return response

tools = [llm_search, internet_search]

In [3]:
from langgraph.prebuilt import create_react_agent

system_message = """
                    You are a helpful chatbot. Who answers with facts only. 
                    Thought: Do I need to use a tool? Yes

                    Action: the action to take, should be either tool or END

                    Action Input: the input to the action

                    Observation: the result of the action

                    Always follow this pattern to get the best results.
                    """

langgraph_agent_executor = create_react_agent(llm, tools, state_modifier=system_message)


response = langgraph_agent_executor.invoke(
        {
            "messages": ["Who is daughter of current President of the USA?"],
        })

print(response["messages"][-1].content)

The current President of the USA is Joe Biden, and his daughters are Ashley Biden and Naomi Biden.
